# Agenda

1. Pandas, NumPy, and PyArrow
2. Arrow -- what it is
3. PyArrow for file loading/saving
4. PyArrow as a back-end data type
5. Transitioning to PyArrow

# Pandas, NumPy, and PyArrow

NumPy has been around for about 20 years. It gives us the speed and efficiency of C data types, but a Python API. That means we can use Python for numeric computing.

NumPy